# Spotify API Exploration 

Get familiar with Spotify API operations in preparation for a larger recommendation system project

Desired Functionalities:
- Acess user-specific data 

## Spotify Client WIP

In [1]:
import os 
import urllib.parse
import random 
import string 
import requests 
from requests.auth import HTTPBasicAuth
import pandas as pd 

class SpotifyClient: 
    """
    Client to wrap all Spotify API operation in for ease of use
    """

    def __init__(self, client_id, client_secret, redirect_uri):
        self.client_id=client_id
        self.client_secret=client_secret
        self.redirect_uri=redirect_uri
        self.access_token=None
        self.refresh_token=None

    def authorize_client(self, scopes):
        """
        Authorize the client to perform actions on a users behalf

        TODO: Add some logic that checks if a refresh token is available to avoid authenticating fully every time

        Args: 
            - scopes (str): List of space-separated operations the application is allowed to perform on the users behalf

        Returns: 
            - access_token (str)
            - refresh_token (str)
        """

        # construct an auth url for the user 
        state = "".join(random.choice(string.ascii_letters + string.digits) for _ in range(16))
        query_params = {
            "response_type": "code", 
            "client_id": self.client_id, 
            "scope": scopes, 
            "redirect_uri": self.redirect_uri, 
            "state": state
        }
        auth_url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(query_params)}"

        # prompt user to visit url
        print("Authenticate at the following URL making sure to copy the URL you are redirected to for the next step.")
        print(f"- {auth_url}")

        # retrieve redirect URL and parse params 
        redirect_url = input("Paste the redirect URL from the previous step here: ")
        parsed_url = urllib.parse.urlparse(redirect_url)
        url_params = urllib.parse.parse_qs(parsed_url.query)

        # get tokens 
        if url_params["state"][0] != state:
            print("Invalid state.")
        else: 
            res = requests.post(
                url="https://accounts.spotify.com/api/token",
                headers={
                    "content-type": "application/x-www-form-urlencoded"
                },
                data={
                    "grant_type": "authorization_code", 
                    "code": url_params["code"][0], 
                    "redirect_uri": os.getenv("SP_REDIRECT_URI")
                }, 
                auth=HTTPBasicAuth(
                    username=os.getenv("SP_CLIENT_ID"),
                    password=os.getenv("SP_CLIENT_SECRET")
                )
            )
            data = res.json()

            # set attributes 
            self.access_token = data["access_token"]
            self.refresh_token = data["refresh_token"]

            return {
                "status_code": 200, 
                "message": "client authorization successful!"
            }
        
    def refresh_token(self):
        res = requests.post(
            url="https://accounts.spotify.com/api/token", 
            headers={
                "Content-Type": "application/x-www-form-urlencoded"
            }, 
            data={
                "grant_type": "refresh_token", 
                "refresh_token": self.refresh_token
            }, 
            auth=HTTPBasicAuth(
                username=self.client_id,
                password=self.client_secret
            )
        )
        return res.json()["refresh_token"]
    
    def read_user_library(self):
        """
        Retreive Spotify IDs for all items a user has saved to their library 

        Returns:
            - spotify_ids (list): List of a user's saved track's Spotify IDs
        """
        # submit first request 
        res = requests.get(
            url="https://api.spotify.com/v1/me/tracks?limit=50", 
            headers={
                "Authorization": f'Bearer {self.access_token}'
            }
        )
        # convert to json 
        data = res.json()
        # get next link 
        next_url = data["next"]
        # instantiate list to hold ids 
        spotify_ids = []
        # move through available items 
        while True:
            # get spotify ids for each of the 50 items returned
            for item in data["items"]:
                spotify_id = item["track"]["id"]
                spotify_ids.append(spotify_id)
            # break out of the loop if no next_url is found
            if not next_url:
                break
            # submit another request using the extracted next url
            res = requests.get(
                url=next_url, 
                headers={
                    "Authorization": f"Bearer {self.access_token}"
                }
            )
            # extract data and next url 
            data = res.json()
            next_url = data["next"]    
        return spotify_ids
    
    def get_audio_features(self, ids):
        """
        Get audio features for a given track using provided Spotify IDs

        Args: 
            - ids (list): List of Spotify IDs

        Returns:
            - features_df (pd.DataFrame): DataFrame of track features
        """
        
        # set initial start and end indices
        start_idx = 0
        end_idx = 100
        # instantaite dataframe to hold features
        features_df = pd.DataFrame()
        # loop over ids and get their audio features
        while start_idx < len(ids):
            # create subset of 100 ids 
            sub_ids = ids[start_idx:end_idx]
            # convert to string for api 
            str_ids = ",".join(sub_ids)
            # submit request for track features
            res = requests.get(
                url=f"https://api.spotify.com/v1/audio-features?ids={str_ids}", 
                headers={
                    "Authorization": f'Bearer {self.access_token}'
                }
            )
            # extract features
            features = res.json()["audio_features"]
            # convert to df 
            df = pd.DataFrame(features)
            # stack into big dataframe 
            features_df = pd.concat([features_df, df])
            # update start and end values
            start_idx = end_idx
            end_idx = start_idx + 100
            # if were at the end, set end index to the final item 
            if end_idx > len(ids):
                end_idx = len(ids)
        return features_df
    
    def get_track_info(self, id):
        """
        Request track information from Spotify API
        """

        # submit api request
        res = requests.get(
            url=f"https://api.spotify.com/v1/tracks/{id}", 
            headers={
                "Authorization": f"Bearer {self.access_token}"
            }
        )

        # extract data
        data = res.json()

        return data

In [68]:
# instantiate client
sp = SpotifyClient(
    client_id=os.getenv("SP_CLIENT_ID"), 
    client_secret=os.getenv("SP_CLIENT_SECRET"),
    redirect_uri=os.getenv("SP_REDIRECT_URI")
)

# authenticate
scopes = "user-read-private user-read-email user-read-recently-played user-library-read"
res = sp.authorize_client(scopes=scopes)

Authenticate at the following URL making sure to copy the URL you are redirected to for the next step.
- https://accounts.spotify.com/authorize?response_type=code&client_id=fd76fa43d59e433b89d63e1ad594f8bb&scope=user-read-private+user-read-email+user-read-recently-played+user-library-read&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&state=uyWMNV742aoo66kS


## Client Testing 

In [72]:
import datetime

def _get_unix_timestamp(mm_dd_yy):
    date = datetime.datetime.strptime(mm_dd_yy, "%m_%d_%y")
    timestamp = int(date.timestamp())
    return timestamp
    
def get_listening_history(start_date):
    """
    Retreive Spotify IDs for all items a user listened to starting at the provided
    date up to the current day 

    Args:
        - start_date (str): Start date in the format MM/DD/YY 

    Returns:
        - spotify_ids (list): List of a user's saved track's Spotify IDs
    """

    # get timestamp 
    unix_timestamp = _get_unix_timestamp(start_date)

    # submit first request 
    res = requests.get(
        url=f"https://api.spotify.com/v1/me/player/recently-played?limit=50&after={unix_timestamp}", 
        headers={
            "Authorization": f'Bearer {sp.access_token}'
        }
    )
    # convert to json 
    data = res.json()
    # get next link 
    next_url = data["next"]
    # instantiate list to hold ids 
    spotify_ids = []
    # move through available items 
    while True:
        # get spotify ids for each of the 50 items returned
        for item in data["items"]:
            spotify_id = item["track"]["id"]
            spotify_ids.append(spotify_id)
        # break out of the loop if no next_url is found
        if not next_url:
            break
        # submit another request using the extracted next url
        res = requests.get(
            url=next_url, 
            headers={
                "Authorization": f"Bearer {sp.access_token}"
            }
        )
        # extract data and next url 
        data = res.json()
        next_url = data["next"]    
    return spotify_ids

In [73]:
get_listening_history(start_date="07_01_24")

{'items': [{'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7GlBOeep6PqTfFi59PTUUN'}, 'href': 'https://api.spotify.com/v1/artists/7GlBOeep6PqTfFi59PTUUN', 'id': '7GlBOeep6PqTfFi59PTUUN', 'name': 'Chappell Roan', 'type': 'artist', 'uri': 'spotify:artist:7GlBOeep6PqTfFi59PTUUN'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW'

['1k2pQc5i348DCHwbn5KTdc',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 '6SpLc7EXZIPpy0sVko0aoU',
 '1ZbGW21L7nKBdMyyrpzJPA',
 '5R6Q2E0wC10LDfrLLcKtwD',
 '6QnUjSX3r96w2vAyF6H4QM',
 '0sbRcQmCZGT7Kzym7OBoxi',
 '2uqYupMHANxnwgeiXTZXzd',
 '5SOYmkRawOJrpufdWsAfo8',
 '50nfwKoDiSYg8zOCREWAm5',
 '01oPNCtTniFT3YM4K3ksTf',
 '1zZIGnXIcDKU8QIUkR20nK',
 '100eDEmpWV5YGVCqHI0leU',
 '0pI6RwmnFl9CawvPpvMVzO',
 '6u0x5ad9ewHvs3z6u9Oe3c',
 '3rgTS3933lMWoPiN6CW4qY',
 '1rqqCSm0Qe4I9rUvWncaom',
 '7qTaDOcld0VmBWXnkbUj45',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 '1SjsVdSXpwm1kTdYEHoPIT',
 '2A7EJohkfEjjCtnl44hvcJ',
 '0sbRcQmCZGT7Kzym7OBoxi',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 '6SpLc7EXZIPpy0sVko0aoU',
 '4KyZLFDRAVvs2naDwADVTJ',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 '6QnUjSX3r96w2vAyF6H4QM',
 '5svL1PkWw8hPEAOdTNLexy',
 '1k2pQc5i348DCHwbn5KTdc',
 '5R6Q2E0wC10LDfrLLcKtwD',
 '18vXApRmJSgQ6wG2ll9AOg',
 '1vxjmNMsJRLEhoYGpOXOXB',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 '7fveJ3pk1eSfxBdVkzdXZ0',
 